Training resnet model

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision
import pickle
import os
import numpy as np

In [44]:
#some constants
import os
print(os.listdir("../data/v2-plant-seedlings-dataset/"))

In [35]:
def create_model():
    
    #load a pretrained resnet model
    res = torchvision.models.resnet50(pretrained=True)
    
    #freeze model weights
    for param in res.parameters():
        param.requires_grad = False
    
    #counting in-features for fully connected layer
    n_inputs = res.fc.in_features
    
    #create fully connected layer with 12 out features + activation layer + softmax
    res.fc = nn.Sequential(nn.Linear(n_inputs, 512),
                          nn.ReLU(),
                          nn.Linear(512, CAT_CNT),
                          nn.ReLU(),
                          nn.LogSoftmax(dim = 1))
    
    return res
    

In [36]:
model = create_model()
print(summary(model, (3, 128, 128)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

In [42]:
def create_dataloaders():
    #unpickling the data files
    #files are trainX_128, trainY_128, validX_128, validY_128
    data_path = os.path.join(".", "balanced_pickled")
    trainX = pickle.load(open(os.path.join(data_path, "trainX_128" ), "rb"))
    trainY = pickle.load(open(os.path.join(data_path, "trainY_128" ), "rb"))
    validX = pickle.load(open(os.path.join(data_path, "validX_128" ), "rb"))
    validY = pickle.load(open(os.path.join(data_path, "validY_128" ), "rb"))
    print(len(trainX))
    print(len(validX))
    #generate data from the pickled np datasets,transforming to torch tensors
    trainX = np.transpose(trainX, (0,3,2,1))
    validX = np.transpose(validX, (0,3,2,1))
    
    tensor_trainX = Variable(torch.from_numpy(np.array(trainX)).float(), requires_grad=False)
    tensor_trainY = Variable(torch.from_numpy(np.array(trainY)).long(), requires_grad=False)
    
    train = TensorDataset(tensor_trainX, tensor_trainY)
    trainLoader = {DataLoader(train, batch_size = 20, shuffle = True)}

    tensor_validX = torch.stack([torch.Tensor(i) for i in validX])
    tensor_validY = torch.stack([torch.Tensor(i) for i in validY])
    valid = TensorDataset(tensor_validX, tensor_validY)
    validLoader = { DataLoader(valid)}
    
    return (trainLoader, validLoader)


In [43]:
create_dataloaders()

2732
304


({<torch.utils.data.dataloader.DataLoader at 0x14a949a2630>},
 {<torch.utils.data.dataloader.DataLoader at 0x14a949a2470>})

In [49]:
def train_model(model):   
    running_loss = 0
    running_corrects = 0
    phase = 'train'
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    (t_loader, v_loader) = create_dataloaders()
    
    epochs = 3
    steps = 0
    print_every = 1
    train_losses, test_losses = [], []
    
    for epoch in range (epochs):
        print('Epoch{}/{}.'.format(epoch, epochs-1))
        print('-' * 10)
        
        running_loss = 0.0
        
        tl = next(iter(t_loader))
        for i, (inputs, labels) in enumerate(tl):
            inputs= inputs.to(device)
            labels = labels.to(device)
            
            steps +=1
            
            #clears the gradients of all optimized tensors
            optimizer.zero_grad()
            
            #forwards + backwards + optimize
            outputs = model.forward(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()


            # print statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == torch.max(labels, 1)[1])
        
        epoch_loss = running_loss/TRAIN_SIZE
        epoch_corrects = running_corrects.double() / TRAIN_SIZE
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_corrects.double()))
            
        print()
        
    #running_loss = 0.0
    #save the model
    torch.save(model.state_dict(), os.path.join(".", "models"))

In [50]:
train_model(model)

2732
304
Epoch0/2.
----------
train Loss: 1.7382 Acc: 0.5106

Epoch1/2.
----------
train Loss: 1.5382 Acc: 1.0597

Epoch2/2.
----------
train Loss: 1.3895 Acc: 1.6563



PermissionError: [Errno 13] Permission denied: '.\\models'